<a href="https://colab.research.google.com/github/AIsquare/RAG/blob/main/RAG_Evaluation_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain openai weaviate-client ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.9/324.9 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [3]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [28]:
url="https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"

In [29]:
res = requests.get(url)


In [30]:
res.text

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [31]:
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

In [32]:
loader=TextLoader("/content/state_of_the_union.txt")

In [33]:
documents=loader.load()


In [34]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [35]:
chunks=text_splitter.split_documents(documents)

In [36]:
chunks

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.', metadata={'source': '/content/state_of_the_union.txt'}),
 Document(page_content='Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessnes

In [37]:
len(chunks)


90

In [38]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

In [40]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [41]:
import os
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [42]:
client=weaviate.Client(
    embedded_options=EmbeddedOptions()
)

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.7/weaviate-v1.23.7-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 4012


In [43]:
vectorstore=Weaviate.from_documents(
    client=client,
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    by_text=False

)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [44]:
retriever=vectorstore.as_retriever()

In [45]:
from langchain.chat_models import ChatOpenAI

In [46]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [47]:
llm=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.2)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [48]:
# Define prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [49]:
prompt = ChatPromptTemplate.from_template(template)


In [50]:
rag_chain=(
    {"context":retriever, "question":RunnablePassthrough()}
          | prompt
          | llm
          | StrOutputParser()
)

In [51]:
res.text

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [52]:

questions=["what did the President say about Justic Breyer?",
           "What did the President say abou Intel's CEO?",
           "What did the President say about gun violence?"

           ]

In [53]:
groud_truths=[["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service."],
              ["The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion."],
              ["The president asked Congress to pass proven measures to reduce gun violence."]]

In [54]:
query

NameError: name 'query' is not defined

In [ ]:
rag_chain.invoke(query)

'The President honored Justice Breyer for his service and mentioned nominating Judge Ketanji Brown Jackson to continue his legacy.'

In [56]:
answer=[]
context=[]
for query in questions:
  answer.append(rag_chain.invoke(query))
  context.append([docs.page_content  for docs in retriever.get_relevant_documents(query)])




/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [57]:
answer

['The President honored Justice Stephen Breyer for his service and legacy of excellence.',
 "The President mentioned that Intel's CEO, Pat Gelsinger, is ready to increase their investment from $20 billion to $100 billion.",
 'The President called for Congress to pass measures to reduce gun violence, including universal background checks and banning assault weapons and high-capacity magazines. He emphasized the need to repeal the liability shield for gun manufacturers and highlighted that these laws save lives.']

In [61]:
context

[['Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.',
  'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
  'A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Borde

In [58]:
# To dict
data = {
    "question": questions,
    "answer": answer,
    "contexts": context,
    "ground_truths": groud_truths
}




In [59]:
from datasets import Dataset
dataset=Dataset.from_dict(data)

In [60]:
!pip install Dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.30
    Uninstalling SQLAlchemy-2.0.30:
      Successfully uninstalled SQLAlchemy-2.0.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.


In [62]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [63]:
result=evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy
    ]
)

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

In [64]:
result

{'context_precision': 1.0000, 'context_recall': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.8893}

In [65]:
df=result.to_pandas()

In [66]:
df

,question,answer,contexts,ground_truths,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,what did the President say about Justic Breyer?,The President honored Justice Stephen Breyer f...,"[Tonight, I’d like to honor someone who has de...",[The president said that Justice Breyer has de...,The president said that Justice Breyer has ded...,1.0,1.0,1.0,0.812977
1,What did the President say abou Intel's CEO?,"The President mentioned that Intel's CEO, Pat ...",[But that’s just the beginning. \n\nIntel’s CE...,[The president said that Pat Gelsinger is read...,The president said that Pat Gelsinger is ready...,1.0,1.0,1.0,0.941490
2,What did the President say about gun violence?,The President called for Congress to pass meas...,[And I ask Congress to pass proven measures to...,[The president asked Congress to pass proven m...,The president asked Congress to pass proven me...,1.0,1.0,1.0,0.913553


In [67]:
df.iloc[0,:]

question               what did the President say about Justic Breyer?
answer               The President honored Justice Stephen Breyer f...
contexts             [Tonight, I’d like to honor someone who has de...
ground_truths        [The president said that Justice Breyer has de...
ground_truth         The president said that Justice Breyer has ded...
context_precision                                                  1.0
context_recall                                                     1.0
faithfulness                                                       1.0
answer_relevancy                                              0.812977
Name: 0, dtype: object

In [68]:
#data
#this is my actual query
df.iloc[0,:].question

'what did the President say about Justic Breyer?'

In [69]:
#predicted
#generation of the model
df.iloc[0,:].answer

'The President honored Justice Stephen Breyer for his service and legacy of excellence.'

In [70]:
#actual
#actual answer(manually we have written this answer)(this could be llm generated)
df.iloc[0,:].ground_truth

'The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service.'

In [71]:
#average value
#retrieval result(context)
df.iloc[0,:].contexts[0]

'Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.'

In [72]:
context_precision                                                  1.0
context_recall                                                     1.0
faithfulness                                                       1.0
answer_relevancy                                               0.83073

SyntaxError: invalid syntax (<ipython-input-72-1b1b4c303f56>, line 1)

In [ ]:
raga:https://docs.ragas.io/en/latest/concepts/metrics/context_recall.html
TrueLence:https://www.trulens.org/trulens_eval/getting_started/core_concepts/rag_triad/
ares:https://github.com/stanford-futuredata/ARES?tab=readme-ov-file#section3

In [ ]:
#rouge summrization
#bleu translation

In [73]:
from datasets import Dataset
from ragas.metrics import answer_similarity
from ragas import evaluate


data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[answer_similarity])
score.to_pandas()

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

,question,answer,ground_truth,answer_similarity
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on January 15, 1967",0.996378
1,Who won the most super bowls?,The most super bowls have been won by The New ...,The New England Patriots have won the Super Bo...,0.924313


In [74]:
from datasets import Dataset
from ragas.metrics import faithfulness, answer_correctness
from ragas import evaluate

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[answer_correctness])
score.to_pandas()

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

,question,answer,ground_truth,answer_correctness
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on January 15, 1967",0.749095
1,Who won the most super bowls?,The most super bowls have been won by The New ...,The New England Patriots have won the Super Bo...,0.731078
